# Training a Torch Image Classifier

This tutorial shows you how to train an image classifier using the [Ray AI Runtime](air) (AIR).

You should be familiar with [PyTorch](https://pytorch.org/) before starting the tutorial. If you need a refresher, read PyTorch's [training a classifier](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) tutorial.

## Before you begin

* Install the [Ray AI Runtime](air). You need Ray 2.0 or later to run this example.

In [1]:
# !pip install 'ray[air]'

* Install `requests`, `torch`, and `torchvision`.

In [2]:
# !pip install requests torch torchvision

## Load and normalize CIFAR-10

We'll train our classifier on a popular image dataset called [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html).

First, let's load CIFAR-10 into a Ray Dataset.

In [3]:
import ray
from ray.data.datasource import SimpleTorchDatasource
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)


def train_dataset_factory():
    return torchvision.datasets.CIFAR10(
        root="./data", download=True, train=True, transform=transform
    )


def test_dataset_factory():
    return torchvision.datasets.CIFAR10(
        root="./data", download=True, train=False, transform=transform
    )


train_dataset: ray.data.Dataset = ray.data.read_datasource(
    SimpleTorchDatasource(), dataset_factory=train_dataset_factory
)
test_dataset: ray.data.Dataset = ray.data.read_datasource(
    SimpleTorchDatasource(), dataset_factory=test_dataset_factory
)

2022-10-28 10:33:14,804	INFO worker.py:1524 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2022-10-28 10:33:20,234	WARNING read_api.py:297 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
(_get_read_tasks pid=30204) 2022-10-28 10:33:20,231	WARNING torch_datasource.py:56 -- `SimpleTorchDatasource` doesn't support parallel reads. The `parallelism` argument will be ignored.


(_execute_read_task_nosplit pid=30204) Files already downloaded and verified


2022-10-28 10:33:37,209	WARNING read_api.py:297 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
(_get_read_tasks pid=30204) 2022-10-28 10:33:37,207	WARNING torch_datasource.py:56 -- `SimpleTorchDatasource` doesn't support parallel reads. The `parallelism` argument will be ignored.


(_execute_read_task_nosplit pid=30204) Files already downloaded and verified


In [4]:
train_dataset

{py:class}`SimpleTorchDatasource <ray.data.datasource.SimpleTorchDatasource>` doesn't parallelize reads, so you shouldn't use it with larger datasets.

Next, let's represent our data using a dictionary of ndarrays instead of tuples. This lets us call {py:meth}`Dataset.iter_torch_batches <ray.data.Dataset.iter_torch_batches>` later in the tutorial.

In [5]:
from typing import Dict, Tuple
import numpy as np
import torch


def convert_batch_to_numpy(batch: Tuple[torch.Tensor, int]) -> Dict[str, np.ndarray]:
    images = np.array([image.numpy() for image, _ in batch])
    labels = np.array([label for _, label in batch])
    return {"image": images, "label": labels}


train_dataset = train_dataset.map_batches(convert_batch_to_numpy)
test_dataset = test_dataset.map_batches(convert_batch_to_numpy)

Read->Map_Batches:   0%|                                                                                                      | 0/1 [00:00<?, ?it/s]

(_map_block_nosplit pid=30204) Files already downloaded and verified


Read->Map_Batches:   0%|                                                                                                      | 0/1 [00:00<?, ?it/s]

(_map_block_nosplit pid=30204) Files already downloaded and verified


Read->Map_Batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.56s/it]


In [6]:
train_dataset

## Train a convolutional neural network

Now that we've created our datasets, let's define the training logic.

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

We define our training logic in a function called `train_loop_per_worker`. This function contains regular PyTorch code with a few notable exceptions:
* We wrap our model with {py:func}`train.torch.prepare_model <ray.train.torch.prepare_model>`.
* We call {py:func}`session.get_dataset_shard <ray.air.session.get_dataset_shard>` and {py:meth}`Dataset.iter_torch_batches <ray.data.Dataset.iter_torch_batches>` to get a subset of our training data.
* We save model state using {py:func}`session.report <ray.air.session.report>`.

In [8]:
from ray import train
from ray.air import session, Checkpoint
from ray.train.torch import TorchCheckpoint
import torch.nn as nn
import torch.optim as optim
import torchvision


def train_loop_per_worker(config):
    model = train.torch.prepare_model(Net())

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    train_dataset_shard = session.get_dataset_shard("train")

    for epoch in range(2):
        running_loss = 0.0
        train_dataset_batches = train_dataset_shard.iter_torch_batches(
            batch_size=config["batch_size"],
        )
        for i, batch in enumerate(train_dataset_batches):
            # get the inputs and labels
            inputs, labels = batch["image"], batch["label"]

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")
                running_loss = 0.0

        metrics = dict(running_loss=running_loss)
        checkpoint = TorchCheckpoint.from_state_dict(model.module.state_dict())
        session.report(metrics, checkpoint=checkpoint)

Finally, we can train our model. This should take a few minutes to run.

In [9]:
from ray.train.torch import TorchTrainer
from ray.air.config import ScalingConfig

trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={"batch_size": 2},
    datasets={"train": train_dataset},
    scaling_config=ScalingConfig(num_workers=2),
)
result = trainer.fit()
latest_checkpoint = result.checkpoint

(RayTrainWorker pid=30317) 2022-10-28 10:34:01,570	INFO config.py:88 -- Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=30317) 2022-10-28 10:34:06,601	INFO train_loop_utils.py:292 -- Moving model to device: cpu
(RayTrainWorker pid=30317) 2022-10-28 10:34:06,608	INFO train_loop_utils.py:352 -- Wrapping provided model in DistributedDataParallel.


(RayTrainWorker pid=30318) [1,  2000] loss: 2.255
(RayTrainWorker pid=30317) [1,  2000] loss: 2.256
(RayTrainWorker pid=30318) [1,  4000] loss: 1.952
(RayTrainWorker pid=30317) [1,  4000] loss: 1.928
(RayTrainWorker pid=30318) [1,  6000] loss: 1.730
(RayTrainWorker pid=30317) [1,  6000] loss: 1.737
(RayTrainWorker pid=30318) [1,  8000] loss: 1.633
(RayTrainWorker pid=30317) [1,  8000] loss: 1.655
(RayTrainWorker pid=30318) [1, 10000] loss: 1.595
(RayTrainWorker pid=30317) [1, 10000] loss: 1.540
(RayTrainWorker pid=30318) [1, 12000] loss: 1.501
(RayTrainWorker pid=30317) [1, 12000] loss: 1.495


Trial name,_time_this_iter_s,_timestamp,_training_iteration,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,running_loss,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
TorchTrainer_b1c4e_00000,31.5717,1666978510,2,2022-10-28_10-35-10,True,,b0bdb4249fa14364aff36772b006aa15,0,Kais-MBP.local.meter,2,127.0.0.1,30301,621.189,True,72.1703,31.5704,72.1703,1666978510,0,,2,b1c4e_00000,0.175393


(RayTrainWorker pid=30318) [2,  2000] loss: 1.443
(RayTrainWorker pid=30317) [2,  2000] loss: 1.463
(RayTrainWorker pid=30318) [2,  4000] loss: 1.437
(RayTrainWorker pid=30317) [2,  4000] loss: 1.414
(RayTrainWorker pid=30318) [2,  6000] loss: 1.423
(RayTrainWorker pid=30317) [2,  6000] loss: 1.398
(RayTrainWorker pid=30318) [2,  8000] loss: 1.386
(RayTrainWorker pid=30317) [2,  8000] loss: 1.404
(RayTrainWorker pid=30318) [2, 10000] loss: 1.366
(RayTrainWorker pid=30317) [2, 10000] loss: 1.328
(RayTrainWorker pid=30318) [2, 12000] loss: 1.312
(RayTrainWorker pid=30317) [2, 12000] loss: 1.315


2022-10-28 10:35:13,436	INFO tune.py:788 -- Total run time: 79.11 seconds (78.62 seconds for the tuning loop).


To scale your training script, create a [Ray Cluster](cluster-index) and increase the number of workers. If your cluster contains GPUs, add `"use_gpu": True` to your scaling config.

```{code-block} python
scaling_config=ScalingConfig(num_workers=8, use_gpu=True)
```

## Test the network on the test data

Let's see how our model performs.

To classify images in the test dataset, we'll need to create a {py:class}`Predictor <ray.train.predictor.Predictor>`.

{py:class}`Predictors <ray.train.predictor.Predictor>` load data from checkpoints and efficiently perform inference. In contrast to {py:class}`TorchPredictor <ray.train.torch.TorchPredictor>`, which performs inference on a single batch, {py:class}`BatchPredictor <ray.train.batch_predictor.BatchPredictor>` performs inference on an entire dataset. Because we want to classify all of the images in the test dataset, we'll use a {py:class}`BatchPredictor <ray.train.batch_predictor.BatchPredictor>`.

In [10]:
from ray.train.torch import TorchPredictor
from ray.train.batch_predictor import BatchPredictor

batch_predictor = BatchPredictor.from_checkpoint(
    checkpoint=latest_checkpoint,
    predictor_cls=TorchPredictor,
    model=Net(),
)

outputs: ray.data.Dataset = batch_predictor.predict(
    data=test_dataset,
    dtype=torch.float,
    feature_columns=["image"],
    keep_columns=["label"],
)

Map Progress (1 actors 1 pending): 100%|██████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.76s/it]


Our model outputs a list of energies for each class. To classify an image, we
choose the class that has the highest energy.

In [11]:
import numpy as np


def convert_logits_to_classes(df):
    best_class = df["predictions"].map(lambda x: x.argmax())
    df["prediction"] = best_class
    return df[["prediction", "label"]]


predictions = outputs.map_batches(convert_logits_to_classes)

predictions.show(1)

Map_Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 22.35it/s]

{'prediction': 3, 'label': 3}


Now that we've classified all of the images, let's figure out which images were
classified correctly. The ``predictions`` dataset contains predicted labels and 
the ``test_dataset`` contains the true labels. To determine whether an image 
was classified correctly, we join the two datasets and check if the predicted 
labels are the same as the actual labels.

In [12]:
def calculate_prediction_scores(df):
    df["correct"] = df["prediction"] == df["label"]
    return df


scores = predictions.map_batches(calculate_prediction_scores)

scores.show(1)

Map_Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 48.41it/s]

{'prediction': 3, 'label': 3, 'correct': True}


To compute our test accuracy, we'll count how many images the model classified 
correctly and divide that number by the total number of test images.

In [13]:
scores.sum(on="correct") / scores.count()

Shuffle Reduce: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 127.89it/s]


0.5359

## Deploy the network and make a prediction

Our model seems to perform decently, so let's deploy the model to an 
endpoint. This allows us to make predictions over the Internet.

In [14]:
from ray import serve
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import json_to_ndarray


serve.run(
    PredictorDeployment.bind(
        TorchPredictor,
        latest_checkpoint,
        model=Net(),
        http_adapter=json_to_ndarray,
    )
)

(ServeController pid=30454) INFO 2022-10-28 10:35:19,465 controller 30454 http_state.py:132 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-975942d2ba1f05aeb09954e04c3db4001be784320aa5d5c25491f28a' on node '975942d2ba1f05aeb09954e04c3db4001be784320aa5d5c25491f28a' listening on '127.0.0.1:8000'
(HTTPProxyActor pid=30462) INFO:     Started server process [30462]
(ServeController pid=30454) INFO 2022-10-28 10:35:21,229 controller 30454 deployment_state.py:1311 - Adding 1 replica to deployment 'PredictorDeployment'.


RayServeSyncHandle(deployment='PredictorDeployment')

Let's classify a test image.

In [15]:
image = test_dataset.take(1)[0]["image"]

You can perform inference against a deployed model by posting a dictionary with an `"array"` key. To learn more about the default input schema, read the {py:class}`NdArray <ray.serve.http_adapters.NdArray>` documentation.

In [16]:
import requests

payload = {"array": image.tolist(), "dtype": "float32"}
response = requests.post("http://localhost:8000/", json=payload)
response.json()

[-0.8589608669281006,
 -1.885204792022705,
 0.6682103872299194,
 1.633802890777588,
 0.5621041059494019,
 0.8943464159965515,
 0.4174749553203583,
 0.16365858912467957,
 -0.4701939523220062,
 -1.167943000793457]

(HTTPProxyActor pid=30462) INFO 2022-10-28 10:35:24,287 http_proxy 127.0.0.1 http_proxy.py:320 - POST / 200 22.7ms
(ServeReplica:PredictorDeployment pid=30465) INFO 2022-10-28 10:35:24,285 PredictorDeployment PredictorDeployment#tjdWjQ replica.py:510 - HANDLE __call__ OK 18.0ms
